# Clasificación de Texto usando SageMaker BlazingText

La Clasificación de Texto puede ser usada para resolver varios casos de uso como análisis de sentimiento, detección de spam, predicción de hashtags, etc. Este cuaderno demuestra el uso de SageMaker BlazingText para realizar la clasificación supervisada de texto binario/multi-clase con una o varias etiquetas. BlazingText puede entrenar el modelo en más de mil millones de palabras en un par de minutos utilizando una CPU multinúcleo o una GPU, al tiempo que logra un rendimiento a la par con los algoritmos de clasificación de texto de aprendizaje profundo de última generación. BlazingText amplía el clasificador de texto fastText para aprovechar la aceleración de la GPU mediante kernels CUDA personalizados.

### Setup

Vamos a empezar por especificar:

- El bucket de S3 y el prefijo que quieres usar para los datos de entrenamiento y modelo. Esto debería estar dentro de la misma región que la Instancia de Notebook, entrenamiento y alojamiento. Si no especificas un bucket, SageMaker SDK creará un bucket por defecto siguiendo una convención de nomenclatura predefinida en la misma región.

- El rol IAM ARN utilizado para dar acceso a SageMaker a sus datos. Puede ser obtenido usando el método **get_execution_role** de sagemaker python SDK.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
print(
    role
)  # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket()  # Replace with your own bucket name if needed
print(bucket)
prefix = "blazingtext/supervised"  # Replace with the prefix under which you want to store the data if needed

### Preparación de los datos

Ahora descargaremos un conjunto de datos de la web con el que queremos entrenar el modelo de clasificación de texto. BlazingText espera un único archivo de texto preprocesado con tokens separados por espacios y cada línea del archivo debe contener una única frase y la(s) etiqueta(s) correspondiente(s) precedida(s) por "\__label\__".

En este ejemplo, vamos a entrenar el modelo de clasificación de texto en el [DBPedia Ontology Dataset](https://wiki.dbpedia.org/services-resources/dbpedia-data-set-2014#2) tal y como hicieron [Zhang et al](https://arxiv.org/pdf/1509.01626.pdf). El conjunto de datos ontológicos DBpedia se construye eligiendo 14 clases no superpuestas de DBpedia 2014. Tiene 560.000 muestras de entrenamiento y 70.000 muestras de prueba. Los campos que utilizamos para este conjunto de datos contienen el título y el resumen de cada artículo de Wikipedia.

In [ ]:
!wget https://github.com/saurabh3949/Text-Classification-Datasets/raw/master/dbpedia_csv.tar.gz

In [ ]:
!tar -xzvf dbpedia_csv.tar.gz

In [ ]:
!head dbpedia_csv/train.csv -n 3

Como puede verse en el resultado anterior, el CSV tiene 3 campos: índice de etiqueta, título y resumen. En primer lugar, vamos a crear un índice de etiqueta para asignar el nombre de la etiqueta y luego proceder a preprocesar el conjunto de datos para la ingestión por BlazingText.

El siguiente código crea la correspondencia entre los índices enteros y la etiqueta de la clase, que se utilizará posteriormente para recuperar el nombre real de la clase durante la inferencia.

In [ ]:
index_to_label = {}
with open("dbpedia_csv/classes.txt") as f:
    for i, label in enumerate(f.readlines()):
        index_to_label[str(i + 1)] = label.strip()
index_to_label

### Preprocesamiento de datos
Necesitamos preprocesar los datos de entrenamiento en formato de **texto tokenizado separado por espacios** que pueda ser consumido por el algoritmo `BlazingText`. Además, como se ha mencionado anteriormente, las etiquetas de clase deben ir precedidas de `__label__` y deben aparecer en la misma línea que la frase original. Utilizaremos la biblioteca `nltk` para tokenizar las frases de entrada del conjunto de datos DBPedia. 

Descargar el tokenizador nltk y otras bibliotecas

In [ ]:
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv
import nltk

nltk.download("punkt")

In [ ]:
def transform_instance(row):
    cur_row = []
    label = "__label__" + index_to_label[row[0]]  # Prefix the index-ed label with __label__
    cur_row.append(label)
    cur_row.extend(nltk.word_tokenize(row[1].lower()))
    cur_row.extend(nltk.word_tokenize(row[2].lower()))
    return cur_row

* La `transform_instance` se aplicará a cada instancia de datos en paralelo utilizando el módulo de multiprocesamiento de python

In [ ]:
def preprocess(input_file, output_file, keep=1):
    all_rows = []
    with open(input_file, "r") as csvinfile:
        csv_reader = csv.reader(csvinfile, delimiter=",")
        for row in csv_reader:
            all_rows.append(row)
    shuffle(all_rows)
    all_rows = all_rows[: int(keep * len(all_rows))]
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows = pool.map(transform_instance, all_rows)
    pool.close()
    pool.join()

    with open(output_file, "w") as csvoutfile:
        csv_writer = csv.writer(csvoutfile, delimiter=" ", lineterminator="\n")
        csv_writer.writerows(transformed_rows)

In [ ]:
%%time

# Preparing the training dataset

# Since preprocessing the whole dataset might take a couple of mintutes,
# we keep 20% of the training dataset for this demo.
# Set keep to 1 if you want to use the complete dataset
preprocess("dbpedia_csv/train.csv", "dbpedia.train", keep=0.8)

# Preparing the validation dataset
preprocess("dbpedia_csv/test.csv", "dbpedia.validation")

La celda de preprocesamiento de datos puede tardar un minuto en ejecutarse. Después de que el preprocesamiento de datos esté completo, necesitamos subirlo a S3 para que pueda ser consumido por SageMaker para ejecutar trabajos de entrenamiento. Usaremos Python SDK para subir estos dos archivos al bucket y a la ubicación del prefijo que hemos establecido anteriormente.

In [ ]:
%%time

train_channel = prefix + "/train"
validation_channel = prefix + "/validation"

sess.upload_data(path="dbpedia.train", bucket=bucket, key_prefix=train_channel)
sess.upload_data(path="dbpedia.validation", bucket=bucket, key_prefix=validation_channel)

s3_train_data = "s3://{}/{}".format(bucket, train_channel)
s3_validation_data = "s3://{}/{}".format(bucket, validation_channel)

A continuación tenemos que configurar una ubicación de salida en S3, donde se volcará el artefacto modelo. Estos artefactos son también la salida del trabajo de traning del algoritmo.

In [ ]:
s3_output_location = "s3://{}/{}/output".format(bucket, prefix)

### Entrenamiento
Ahora que hemos terminado con toda la configuración necesaria, estamos listos para entrenar nuestro detector de objetos. Para empezar, vamos a crear un objeto ``sageMaker.estimator.Estimator``. Este estimador lanzará el trabajo de entrenamiento.

In [ ]:
region_name = boto3.Session().region_name

In [ ]:
from sagemaker import image_uris
container = sagemaker.image_uris.retrieve(
    region=region_name, 
    framework="blazingtext", 
    version="1"
)
print("Using SageMaker BlazingText container: {} ({})".format(container, region_name))

### Entrenamiento del modelo BlazingText para la clasificación supervisada de textos

De forma similar a la implementación original de [Word2Vec](https://arxiv.org/pdf/1301.3781.pdf), SageMaker BlazingText proporciona una implementación eficiente de las arquitecturas de bolsa continua de palabras (CBOW) y skip-gram utilizando Muestreo Negativo, en CPUs y adicionalmente en GPU[s]. La implementación en GPU utiliza núcleos CUDA altamente optimizados. Para obtener más información, consulte [*BlazingText: Escalado y aceleración de Word2Vec utilizando múltiples GPU*](https://dl.acm.org/citation.cfm?doid=3146347.3146354).

Además de skip-gram y CBOW, SageMaker BlazingText también soporta el modo "Batch Skipgram", que utiliza eficientes operaciones mini-batch y matriz-matriz ([BLAS Level 3 routines](https://software.intel.com/en-us/mkl-developer-reference-fortran-blas-level-3-routines)). Este modo permite el entrenamiento distribuido de word2vec a través de múltiples nodos CPU, permitiendo un escalado casi lineal del cómputo word2vec para procesar cientos de millones de palabras por segundo. Para más información, consulte [*Paralelización de Word2Vec en memoria compartida y distribuida*](https://arxiv.org/pdf/1604.04661.pdf).

BlazingText también admite un modo *supervisado* para la clasificación de textos. Amplía el clasificador de texto FastText para aprovechar la aceleración de la GPU mediante kernels CUDA personalizados. El modelo puede entrenarse con más de mil millones de palabras en un par de minutos utilizando una CPU multinúcleo o una GPU, al tiempo que alcanza un rendimiento equiparable al de los algoritmos de clasificación de texto de aprendizaje profundo más avanzados. Para más información, consulte la [documentación del algoritmo](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html).

En resumen, BlazingText admite los siguientes modos en diferentes tipos de instancias:

|          Modes         	| cbow (supports subwords training) 	| skipgram (supports subwords training) 	| batch_skipgram 	| supervised |
|:----------------------:	|:----:	|:--------:	|:--------------:	| :--------------:	|
|   Single CPU instance  	|   ✔  	|     ✔    	|        ✔       	|  ✔  |
|   Single GPU instance  	|   ✔  	|     ✔    	|                	|  ✔ (Instance with 1 GPU only)  |
| Multiple CPU instances 	|      	|          	|        ✔       	|     | |

Ahora, definamos el `Estimator` de SageMaker con configuraciones de recursos e hiperparámetros para entrenar la Clasificación de Texto en el conjunto de datos *DBPedia*, usando el modo "supervisado" en una instancia `c4.4xlarge`.

Consulte [BlazingText Hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext_hyperparameters.html) en la documentación de Amazon SageMaker para obtener la lista completa de hiperparámetros.

In [ ]:
bt_model = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.c4.4xlarge",
    volume_size=30,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_location,
    hyperparameters={
        "mode": "supervised",
        "epochs": 1,
        "min_count": 2,
        "learning_rate": 0.05,
        "vector_dim": 10,
        "early_stopping": True,
        "patience": 4,
        "min_epochs": 5,
        "word_ngrams": 2,
    },
)

Ahora que los hiper-parámetros están configurados, preparemos el *handshake* entre nuestros canales de datos y el algoritmo. Para ello, tenemos que crear los objetos `sagemaker.session.s3_input` de nuestros canales de datos. Estos objetos se colocan en un diccionario simple, que el algoritmo consume.

In [ ]:
train_data = sagemaker.inputs.TrainingInput(
    s3_train_data,
    distribution="FullyReplicated",
    content_type="text/plain",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    s3_validation_data,
    distribution="FullyReplicated",
    content_type="text/plain",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

Tenemos nuestro objeto `Estimador`, hemos configurado los hiperparámetros para este objeto y tenemos nuestros canales de datos enlazados con el algoritmo. Lo único que queda por hacer es entrenar el algoritmo. El siguiente comando entrenará el algoritmo. El entrenamiento del algoritmo consta de varios pasos. En primer lugar, se aprovisiona la instancia que solicitamos al crear las clases `Estimator` y se configura con las librerías apropiadas. A continuación, los datos de nuestros canales se descargan en la instancia. Una vez hecho esto, comienza el trabajo de entrenamiento. El aprovisionamiento y la descarga de datos llevarán algún tiempo, dependiendo del tamaño de los datos. Por lo tanto, pueden pasar unos minutos antes de que empecemos a obtener los registros de entrenamiento de nuestros trabajos de entrenamiento. Los registros de datos también imprimirán la Precisión en los datos de validación para cada época después de que el trabajo de entrenamiento haya ejecutado `min_epochs`. Esta métrica es un indicador de la calidad del algoritmo. 

Una vez finalizado el trabajo, se imprimirá el mensaje "Trabajo completado". El modelo entrenado se puede encontrar en el bucket de S3 que se configuró como `output_path` en el estimador.

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)

### Alojamiento / Inferencia
Una vez realizado el entrenamiento, podemos desplegar el modelo entrenado como un endpoint alojado en tiempo real de Amazon SageMaker. Esto nos permitirá realizar predicciones (o inferencias) a partir del modelo. Tenga en cuenta que no tenemos que alojar en el mismo tipo de instancia que utilizamos para entrenar. Dado que los puntos finales de instancia estarán en funcionamiento durante mucho tiempo, es aconsejable elegir una instancia más barata para la inferencia.

In [ ]:
from sagemaker.serializers import JSONSerializer

text_classifier = bt_model.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", serializer=JSONSerializer()
)

#### Utilizar el formato JSON para la inferencia

BlazingText soporta `application/json` como tipo de contenido para la inferencia. La carga útil debe contener una lista de sentencias con la clave "**instances**" mientras se pasa al endpoint.

In [ ]:
sentences = [
    "Convair was an american aircraft manufacturing company which later expanded into rockets and spacecraft.",
    "Berwick secondary college is situated in the outer melbourne metropolitan suburb of berwick .",
]

# using the same nltk tokenizer that we used during data preparation for training
tokenized_sentences = [" ".join(nltk.word_tokenize(sent)) for sent in sentences]

payload = {"instances": tokenized_sentences}

response = text_classifier.predict(payload)

predictions = json.loads(response)
print(json.dumps(predictions, indent=2))

Por defecto, el modelo sólo devuelve una predicción, la de mayor probabilidad. Para recuperar las k predicciones más altas, puede establecer `k` en la configuración como se muestra a continuación:

In [ ]:
payload = {"instances": tokenized_sentences, "configuration": {"k": 2}}

response = text_classifier.predict(payload)

predictions = json.loads(response)
print(json.dumps(predictions, indent=2))

### Detener / Cerrar el Endpoint (Opcional)
Por último, deberíamos eliminar el endpoint antes de cerrar el bloc de notas si no necesitamos mantener el endpoint en ejecución para servir predicciones en tiempo real.

In [ ]:
#sess.delete_endpoint(text_classifier.endpoint)

---------------

## Crear un punto final de inferencia asíncrono

Cree un endpoint asíncrono de la misma forma que crearía un endpoint utilizando los servicios de alojamiento de SageMaker:

* Cree un modelo en SageMaker con CreateModel.
* Cree una configuración de endpoint con CreateEndpointConfig.
* Cree un endpoint HTTPS con CreateEndpoint.

Para crear un endpoint, primero se crea un modelo con CreateModel, donde se apunta al artefacto del modelo y a una ruta de registro Docker (Image). A continuación, cree una configuración mediante CreateEndpointConfig, donde especifique uno o más modelos creados mediante la API CreateModel para desplegar y los recursos que desea que SageMaker aprovisione. Cree su punto final con CreateEndpoint utilizando la configuración del punto final especificada en la solicitud. Puede actualizar un endpoint asíncrono con la API UpdateEndpoint. Envíe y reciba solicitudes de inferencia del modelo alojado en el endpoint con InvokeEndpointAsync. Puede eliminar sus puntos finales con la API DeleteEndpoint.

Para una lista completa de las Imágenes SageMaker disponibles, vea [Imágenes de Contenedores de Aprendizaje Profundo Disponibles](https://github.com/aws/deep-learning-containers/blob/master/available_images.md). Vea [Use su propio código de inferencia](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html) para información sobre cómo crear su imagen Docker.

### Crear un modelo
El siguiente ejemplo muestra cómo crear un modelo utilizando el SDK de AWS para Python (Boto3). Las primeras líneas definen

* `sagemaker_client`: Un objeto cliente de bajo nivel de SageMaker que facilita el envío y recepción de solicitudes a los servicios de AWS.
* `sagemaker_role`: Una variable de cadena con el nombre de recurso de Amazon (ARN) del rol IAM de SageMaker.
* `aws_region`: Una variable de cadena con el nombre de tu región de AWS.

In [ ]:
import boto3

# Specify your AWS Region
aws_region='us-east-1'

# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

# Role to give SageMaker permission to access AWS services.
sagemaker_role= "arn:aws:iam::763864518324:role/Lab-AmazonSageMaker-ExecutionRole"

A continuación, especifique la ubicación del modelo preentrenado almacenado en Amazon S3. En este ejemplo, utilizamos un modelo XGBoost preentrenado denominado demo-xgboost-model.tar.gz. La URI completa de Amazon S3 se almacena en una variable de cadena model_url:

In [ ]:
#Create a variable w/ the model S3 URI
s3_bucket = 'sagemaker-us-east-1-763864518324' # Provide the name of your S3 bucket
bucket_prefix='blazingtext'
model_s3_key = f"{bucket_prefix}/supervised/output/blazingtext-2023-04-09-14-04-53-267/output/model.tar.gz"

#Specify S3 bucket w/ model
model_url = f"s3://{s3_bucket}/{model_s3_key}"

Especifique un contenedor primario. Para el contenedor primario, se especifica la imagen Docker que contiene el código de inferencia, los artefactos (del entrenamiento anterior) y un mapa de entorno personalizado que el código de inferencia utiliza cuando se despliega el modelo para las predicciones.

En este ejemplo, especificamos una imagen de contenedor de algoritmo integrado XGBoost:

In [ ]:
from sagemaker import image_uris

# Specify an AWS container image. 
container = image_uris.retrieve(region=aws_region, framework='xgboost', version='0.90-1')

Cree un modelo en Amazon SageMaker con CreateModel. Especifique lo siguiente

* `ModelName`: Un nombre para su modelo (en este ejemplo se almacena como una variable de cadena llamada model_name).

* `ExecutionRoleArn`: El nombre de recurso de Amazon (ARN) del rol de IAM que Amazon SageMaker puede asumir para obtener acceso a los artefactos del modelo y las imágenes de Docker para el despliegue en instancias informáticas de ML o para trabajos de transformación por lotes.

* `PrimaryContainer`: La ubicación de la imagen Docker principal que contiene el código de inferencia, los artefactos asociados y los mapas de entorno personalizados que utiliza el código de inferencia cuando se despliega el modelo para las predicciones.

In [ ]:
model_name = 'my-model'

In [ ]:
#Create model
create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': model_url,
    })

Si está utilizando un contenedor proporcionado por SageMaker, puede aumentar el tiempo de espera del servidor de modelos y los tamaños de la carga útil de los valores predeterminados a los máximos admitidos por el marco estableciendo variables de entorno en este paso. Es posible que no pueda aprovechar los tamaños máximos de tiempo de espera y carga útil que admite Asynchronous Inference si no establece explícitamente estas variables. El siguiente ejemplo muestra cómo puede establecer las variables de entorno para un contenedor PyTorch Inference basado en TorchServe.

In [1]:
create_model_response_torch = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': model_url,
        'Environment': {
            'TS_MAX_REQUEST_SIZE': '100000000',
            'TS_MAX_RESPONSE_SIZE': '100000000',
            'TS_DEFAULT_RESPONSE_TIMEOUT': '1000'
        },
    })

create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': model_url,
        'Environment': {
            'TS_MAX_REQUEST_SIZE': '100000000',
            'TS_MAX_RESPONSE_SIZE': '100000000',
            'TS_DEFAULT_RESPONSE_TIMEOUT': '1000'
        },
    })

Cuando termines de crear tu endpoint, deberías comprobar que has configurado correctamente las variables de entorno imprimiéndolas desde tu script inference.py. La siguiente tabla enumera las variables de entorno de varios frameworks que puedes configurar para cambiar los valores predeterminados.

<table id="w1172aac25c35c11b9c11c33"><thead>
                        <tr>
                            <th>Framework</th>
                            <th>Environment variables</th>
                        </tr>
                    </thead>
                        <tbody><tr>
                            <td>
                                <p>PyTorch 1.8 (based on TorchServe)</p>
                            </td>
                            <td>
                                <p>'TS_MAX_REQUEST_SIZE': '100000000'</p>
                                <p>'TS_MAX_RESPONSE_SIZE': '100000000'</p>
                                <p>'TS_DEFAULT_RESPONSE_TIMEOUT': '1000'</p>
                            </td>
                        </tr>
                        <tr>
                            <td>
                                <p>PyTorch 1.4 (based on MMS)</p>
                            </td>
                            <td>
                                <p>'MMS_MAX_REQUEST_SIZE': '1000000000'</p>
                                <p>'MMS_MAX_RESPONSE_SIZE': '1000000000'</p>
                                <p>'MMS_DEFAULT_RESPONSE_TIMEOUT': '900'</p>
                            </td>
                        </tr>
                        <tr>
                            <td>
                                <p>HuggingFace Inference Container (based on MMS)</p>
                            </td>
                            <td>
                                <p>'MMS_MAX_REQUEST_SIZE': '2000000000'</p>
                                <p>'MMS_MAX_RESPONSE_SIZE': '2000000000'</p>
                                <p>'MMS_DEFAULT_RESPONSE_TIMEOUT': '900'</p>
                            </td>
                        </tr>
                        

In [ ]:
response = client.list_models(
    SortBy='Name',
    SortOrder='Ascending',
    CreationTimeAfter=time_after
)

### Crear una configuración de punto final

Una vez que tenga un modelo, cree una configuración de punto final con CreateEndpointConfig. Los servicios de hospedaje de Amazon SageMaker utilizan esta configuración para desplegar modelos. En la configuración, identifique uno o varios modelos, creados con CreateModel, para implementar los recursos que desea que aprovisione Amazon SageMaker. Especifique el objeto AsyncInferenceConfig y proporcione una ubicación de salida de Amazon S3 para OutputConfig. Opcionalmente, puede especificar temas de Amazon SNS en los que enviar notificaciones sobre los resultados de las predicciones. Para obtener más información sobre los temas de Amazon SNS, consulte Configuración de Amazon SNS.

In [ ]:
import datetime
from time import gmtime, strftime

In [ ]:
# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = f"BlazingTextEndpointConfig-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"

create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": "ml.m5.xlarge", # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            # Location to upload response outputs when no location is provided in the request.
            "S3OutputPath": f"s3://{s3_bucket}/{bucket_prefix}/output",
            # (Optional) specify Amazon SNS topics
            #"NotificationConfig": {
            #    "SuccessTopic": "arn:aws:sns:aws-region:account-id:topic-name",
            #   "ErrorTopic": "arn:aws:sns:aws-region:account-id:topic-name",
            },
        "ClientConfig": {
            # (Optional) Specify the max number of inflight invocations per instance
            # If no value is provided, Amazon SageMaker will choose an optimal value for you
            "MaxConcurrentInvocationsPerInstance": 4
        }
    }
)

print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

En el ejemplo anterior, se especifican las siguientes claves para OutputConfig para el campo AsyncInferenceConfig:

* `S3OutputPath`: Ubicación para cargar las salidas de respuesta cuando no se proporciona ninguna ubicación en la solicitud.
* `NotificationConfig`: (Opcional) Temas SNS que le envían notificaciones cuando una solicitud de inferencia tiene éxito (SuccessTopic) o si falla (ErrorTopic).

También puede especificar el siguiente argumento opcional para ClientConfig en el campo AsyncInferenceConfig:

* `MaxConcurrentInvocationsPerInstance`: (Opcional) El número máximo de solicitudes concurrentes enviadas por el cliente SageMaker al contenedor del modelo.

### Crear punto final

Una vez que tenga su modelo y configuración de punto de enlace, utilice la API CreateEndpoint para crear su punto de enlace. El nombre del punto de enlace debe ser único dentro de una región de AWS en su cuenta de AWS.

A continuación se crea un endpoint utilizando la configuración del endpoint especificada en la solicitud. Amazon SageMaker utiliza el punto de enlace para aprovisionar recursos e implementar modelos.

In [ ]:
# The name of the endpoint.The name must be unique within an AWS Region in your AWS account.
endpoint_name = f"BlazingTextEndpoint-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 


In [ ]:
# Get the endpoint status
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print(f"Endpoint status is {status}")

Cuando llama a la API `CreateEndpoint`, Amazon SageMaker Asynchronous Inference envía una notificación de prueba para comprobar que ha configurado un tema de Amazon SNS. Amazon SageMaker Asynchronous Inference también envía notificaciones de prueba después de las llamadas a `UpdateEndpoint` y `UpdateEndpointWeightsAndCapacities`. Esto permite a SageMaker comprobar que dispone de los permisos necesarios. La notificación puede ser simplemente ignorada. La notificación de prueba tiene la siguiente forma:

In [ ]:
# Create a low-level client representing Amazon SageMaker Runtime
sm_runtime = boto3.client("sagemaker-runtime", region_name=aws_region)

In [ ]:
# Define the input payload for the API
input_data = {'key': 'value'}
payload = json.dumps(input_data)

In [ ]:
# Call the endpoint asynchronously
response = sm_runtime.invoke_endpoint_async(EndpointName=endpoint_name, ContentType='application/json', Body=payload)

In [ ]:
# Get the inference job ID from the response
job_id = response['InferenceId']
print(f"Inference job ID is {job_id}")